Q1: K-Fold Cross Validation for Multiple Linear Regression (Least Square Error Fit)Download the dataset regarding USA House Price Prediction from the following link:https://drive.google.com/file/d/1O_NwpJT-8xGfU_-3llUl2sgPu0xllOrX/view?usp=sharingLoad the dataset and Implement 5- fold cross validation for multiple linear regression(using least square error fit).Steps:<br>a) Divide the dataset into input features (all columns except price) and output variable(price)<br>b) Scale the values of input features.<br>c) Divide input and output features into five folds.<br>d) Run five iterations, in each iteration consider one-fold as test set and remainingfour sets as training set. Find the beta (𝛽) matrix, predicted values, and R2_scorefor each iteration using least square error fit.<br>e) Use the best value of (𝛽) matrix (for which R2_score is maximum), to train theregressor for 70% of data and test the performance for remaining 30% data.

In [21]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold, train_test_split

a) Divide the dataset into input features (all columns except price) and output variable(price)

In [22]:
data = pd.read_csv("USA_Housing.csv")

X = data.drop(columns=['Price'])
y = data['Price']

b) Scale the values of input features.

In [23]:
scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)
print(X_scaled)

[[ 1.02865969 -0.29692705  0.02127433  0.08806222 -1.31759867]
 [ 1.00080775  0.02590164 -0.25550611 -0.72230146  0.40399945]
 [-0.68462915 -0.11230283  1.5162435   0.93084045  0.07240989]
 ...
 [-0.48723454  1.28447022 -2.17026949 -1.50025059 -0.29193658]
 [-0.05459152 -0.44669439  0.14154061  1.18205319  0.65111608]
 [-0.28831272  0.01521477 -0.19434166  0.07185495  1.04162464]]


c) Divide input and output features into five folds.

In [26]:
k_folds = KFold(n_splits=5,shuffle=True,random_state=42)

betas,r2_scores,y_preds = [],[],[]

In [27]:
for train_idx, test_idx in k_folds.split(X_scaled):
    X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
    
    y_train, y_test = y[train_idx], y[test_idx]

    #adding bias term
    X_train_matrix = np.hstack([np.ones((X_train.shape[0],1)),X_train])
    X_test_matrix = np.hstack([np.ones((X_test.shape[0],1)),X_test])

    beta = np.linalg.pinv(X_train_matrix.T @ X_train_matrix)@ X_train_matrix.T @ y_train
    y_cap = X_test_matrix@beta
    r2 = r2_score(y_test,y_cap)
    betas.append(beta)
    r2_scores.append(r2)
    y_preds.append(y_cap)

    print(y_preds)


[array([ 764593.88740947,  947126.10547558,  490147.41553178,
       1202021.72538152,  995799.79963138, 1240643.83197656,
       1371049.80323151,  609160.57630574, 1610587.67996459,
       1443101.22630546, 1071126.32758616,  510258.75520882,
       1124201.97063787,  845281.72330527, 1311860.51923884,
       1261709.43785163, 1051682.47252977, 1539076.97999131,
       1394711.98468397, 1494542.21682855, 1577868.55753656,
        385027.9249513 , 1356640.83892279, 1598770.0790882 ,
       1427009.14145813, 1227292.5524368 , 1544058.05050119,
       1352329.77287414, 1609536.2301316 , 1751850.88845093,
       1193458.84760107, 1488884.78536109, 1430509.55162682,
       1541101.84337795, 1253548.56028956, 1173989.48297491,
       1451580.2341148 , 1425286.31748879,  671601.89587054,
       1282164.4032167 , 1425287.8612071 ,  806952.29885224,
       1439397.74687239, 1424922.73586866, 1180743.31766549,
       1050698.1129626 ,  775297.74280016, 1531208.64375191,
       1631028.42796536

In [28]:
maxIdx = np.argmax(r2_scores)
max_beta = betas[maxIdx]

In [30]:
X_train_70, X_test_30,y_train_70, y_test_30 = train_test_split(X_scaled, y, test_size = 0.3, random_state =42)
X_train2_70 = np.hstack([np.ones((X_train_70.shape[0],1)),X_train_70])
X_test2_30 = np.hstack([np.ones((X_test_30.shape[0],1)),X_test_30])

beta_70 = np.linalg.pinv(X_train2_70.T @ X_train2_70) @ X_train2_70.T @ y_train_70

y_pred_30 = X_test2_30 @ beta_70
r2_30 = r2_score(y_test_30,y_pred_30)

Concept of Validation set for Multiple Linear Regression (Gradient DescentOptimization)<br>Consider the same dataset of Q1, rather than dividing the dataset into five folds, divide thedataset into training set (56%), validation set (14%), and test set (30%).Consider four different values of learning rate i.e. {0.001,0.01,0.1,1}. Compute the values ofregression coefficients for each value of learning rate after 1000 iterations.For each set of regression coefficients, compute R2_score for validation and test set and findthe best value of regression coefficients.

In [33]:
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42)

# for the validation set : we split only the training subset
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)


In [44]:
from sklearn.preprocessing import StandardScaler
import numpy as np

def gradient_descent(X, y, alpha, iterations):
    m, n = X.shape
    weights = np.zeros(n)
    y = y.to_numpy()
    
    for i in range(iterations):
        predictions = X.dot(weights)
        errors = predictions - y
        gradient = (1/m) * X.T.dot(errors)
        weights -= alpha * gradient

        # stop if unstable
        if np.isnan(weights).any() or np.isinf(weights).any():
            print(f"NaN/Inf encountered at iteration {i}")
            break
    return weights

def add_intercept(X):
    intercept = np.ones((X.shape[0], 1))
    return np.hstack((intercept, X))

# --- Correct order: scale first, then add intercept ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled   = scaler.transform(X_val)
X_test_scaled  = scaler.transform(X_test)

X_train_i = add_intercept(X_train_scaled)
X_val_i   = add_intercept(X_val_scaled)
X_test_i  = add_intercept(X_test_scaled)


In [46]:

learning_rates = [0.001, 0.01, 0.1, 1]
iterations = 1000
best_alpha = None
best_weights = None
best_val_r2 = -np.inf

results = []

for alpha in learning_rates:
    weights = gradient_descent(X_train_i, y_train, alpha, iterations)

    # Predictions
    val_pred = X_val_i.dot(weights)
    test_pred = X_test_i.dot(weights)

    # Check for instability
    if np.isnan(val_pred).any() or np.isinf(val_pred).any():
        print(f"Skipping alpha={alpha}: predictions contain NaN/Inf")
        continue

    # Scores
    val_r2 = r2_score(y_val, val_pred)
    test_r2 = r2_score(y_test, test_pred)

    results.append({
        'learning_rate': alpha,
        'weights': weights,
        'val_r2': val_r2,
        'test_r2': test_r2
    })

    # Track best model by validation R2
    if val_r2 > best_val_r2:
        best_val_r2 = val_r2
        best_weights = weights
        best_alpha = alpha


print(f"Best learning rate: {best_alpha}")
print(f"Best validation R2: {best_val_r2}")
print(f"Test R2 at best learning rate: {[r['test_r2'] for r in results if r['learning_rate'] == best_alpha][0]}")
print(f"Best regression coefficients: {best_weights}")


Best learning rate: 0.01
Best validation R2: 0.9098185612062919
Test R2 at best learning rate: 0.9147438859798211
Best regression coefficients: [1232381.36998704  234542.29331285  162395.51342821  121485.70570465
    3089.76469737  151581.05530854]


Pre-processing and Multiple Linear Regression<br>Download the dataset regarding Car Price Prediction from the following link:https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data<br><br>1.Load the dataset with following column names ["symboling", "normalized_losses","make", "fuel_type", "aspiration","num_doors", "body_style", "drive_wheels","engine_location", "wheel_base", "length", "width", "height", "curb_weight","engine_type", "num_cylinders", "engine_size", "fuel_system", "bore", "stroke","compression_ratio", "horsepower", "peak_rpm", "city_mpg", "highway_mpg", "price"]and replace all ? values with NaN<br><br>2. Replace all NaN values with central tendency imputation. Drop the rows with NaNvalues in price column<br><br>3. There are 10 columns in the dataset with non-numeric values. Convert these values tonumeric values using following scheme:<br>(i) For “num_doors” and “num_cylinders”: convert words (number names) to figuresfor e.g., two to 2<br>(ii) For "body_style", "drive_wheels": use dummy encoding scheme<br>(iii) For “make”, “aspiration”, “engine_location”,fuel_type: use label encodingscheme<br>(iv) For fuel_system: replace values containing string pfi to 1 else all values to 0.<br>(v) For engine_type: replace values containing string ohc to 1 else all values to 0.<br><br>4. Divide the dataset into input features (all columns except price) and output variable(price). Scale all input features.<br><br>5. Train a linear regressor on 70% of data (using inbuilt linear regression function ofPython) and test its performance on remaining 30% of data.<br><br>6. Reduce the dimensionality of the feature set using inbuilt PCA decomposition and thenagain train a linear regressor on 70% of reduced data (using inbuilt linear regressionfunction of Python). Does it lead to any performance improvement on test set?

In [54]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.decomposition import PCA

# 1. load dataset
col_names = [
    "symboling", "normalized_losses", "make", "fuel_type", "aspiration",
    "num_doors", "body_style", "drive_wheels", "engine_location", "wheel_base",
    "length", "width", "height", "curb_weight", "engine_type", "num_cylinders",
    "engine_size", "fuel_system", "bore", "stroke", "compression_ratio",
    "horsepower", "peak_rpm", "city_mpg", "highway_mpg", "price"
]

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data"
df = pd.read_csv(url, names=col_names, na_values="?")

# 2. handle missing values (mean for numeric, mode for categorical)
for col in df.columns:
    if df[col].dtype == "object":
        df[col] = df[col].fillna(df[col].mode()[0])
    else:
        df[col] = df[col].fillna(df[col].mean())

# drop rows with NaN price
df.dropna(subset=["price"], inplace=True)
df["price"] = df["price"].astype(float)

# 3. convert categorical → numeric

# (i) map words to numbers
door_map = {"two": 2, "four": 4}
cyl_map = {"two": 2, "three": 3, "four": 4, "five": 5,
           "six": 6, "eight": 8, "twelve": 12}
df["num_doors"] = df["num_doors"].map(door_map).astype(int)
df["num_cylinders"] = df["num_cylinders"].map(cyl_map).astype(int)

# (ii) dummy encoding
df = pd.get_dummies(df, columns=["body_style", "drive_wheels"], drop_first=True)

# (iii) label encoding
for col in ["make", "aspiration", "engine_location", "fuel_type"]:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

# (iv) fuel system → binary
df["fuel_system"] = df["fuel_system"].apply(lambda x: 1 if "pfi" in x else 0)

# (v) engine type → binary
df["engine_type"] = df["engine_type"].apply(lambda x: 1 if "ohc" in x else 0)

# 4. features and target
X = df.drop("price", axis=1)
y = df["price"]

# scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 5. train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42
)

# train regression
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

print("---- Without PCA ----")
print("R2 Score:", r2_score(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))

# 6. PCA (keep 95% variance)
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_scaled)

# split again
Xp_train, Xp_test, yp_train, yp_test = train_test_split(
    X_pca, y, test_size=0.3, random_state=42
)

# regression after PCA
lr_pca = LinearRegression()
lr_pca.fit(Xp_train, yp_train)
yp_pred = lr_pca.predict(Xp_test)

print("---- With PCA ----")
print("R2 Score:", r2_score(yp_test, yp_pred))
print("RMSE:", np.sqrt(mean_squared_error(yp_test, yp_pred)))


---- Without PCA ----
R2 Score: 0.8044422435762588
RMSE: 3663.6361161737345
---- With PCA ----
R2 Score: 0.7500675882701553
RMSE: 4141.771149282544
